In [ ]:
"""dump prophet results to postgres
"""

In [40]:
# import what I need
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy import text
import psycopg2
from psycopg2 import sql
import pandas as pd
from psycopg2.extensions import AsIs


In [41]:
# define folder locations, vars
predictions_names = ['ny_prophet_predictions', 'la_prophet_predictions']
folder = './'

dbname = 'produce_predictions'
username = 'donald'

In [42]:
# engine
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://donald@localhost/produce_predictions


In [43]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [52]:
# now read in each city's predictions - put each in a table in the database
for c in predictions_names:

    predictions = pd.read_csv(folder+c+'.csv', index_col = 0)
    ## insert data into database from Python (proof of concept - this won't be useful for big data, of course)

    predictions.to_sql(c, engine, if_exists='replace')
    
    con = psycopg2.connect(database = dbname, user = username)
# test
sql_query ="""
SELECT "Item", "Current Price", "3 Month Prediction" FROM "ny_prophet_predictions";
"""
query_results=pd.read_sql_query(sql_query,con)
predictions = []
for i in range(0,query_results.shape[0]):
    predictions.append(dict(item=query_results.iloc[i]['Item'], currprice=query_results.iloc[i]['Current Price'], predictprice=query_results.iloc[i]['3 Month Prediction']))
print(predictions)

# test
sql_query ="""
SELECT "Item", "Current Price", "3 Month Prediction" FROM "la_prophet_predictions";
"""
query_results=pd.read_sql_query(sql_query,con)
predictions = []
for i in range(0,query_results.shape[0]):
    predictions.append(dict(item=query_results.iloc[i]['Item'], currprice=query_results.iloc[i]['Current Price'], predictprice=query_results.iloc[i]['3 Month Prediction']))
print(predictions)

con.close()

[{'item': 'APPLES', 'currprice': 27.99, 'predictprice': 32.67}, {'item': 'APRICOTS', 'currprice': 37.46, 'predictprice': 30.16}, {'item': 'ASPARAGUS', 'currprice': 26.14, 'predictprice': 28.85}, {'item': 'AVOCADOS', 'currprice': 33.04, 'predictprice': 56.52}, {'item': 'BANANAS', 'currprice': 16.6, 'predictprice': 16.1}, {'item': 'BEANS', 'currprice': 25.59, 'predictprice': 25.48}, {'item': 'BEETS', 'currprice': 13.21, 'predictprice': 14.01}, {'item': 'BLACKBERRIES', 'currprice': 19.39, 'predictprice': 18.7}, {'item': 'BLUEBERRIES', 'currprice': 30.51, 'predictprice': 16.03}, {'item': 'BROCCOLI', 'currprice': 28.95, 'predictprice': 22.58}, {'item': 'CABBAGE', 'currprice': 12.93, 'predictprice': 13.69}, {'item': 'CANTALOUPS', 'currprice': 21.51, 'predictprice': 13.13}, {'item': 'CARROTS', 'currprice': 15.83, 'predictprice': 18.07}, {'item': 'CAULIFLOWER', 'currprice': 19.67, 'predictprice': 15.72}, {'item': 'CELERY', 'currprice': 18.14, 'predictprice': 16.51}, {'item': 'CHERRIES', 'currp

In [50]:
# now read in each item's data - put each in a table in the database
citydict = {'NEW+YORK':'newyork', 'LOS+ANGELES':'losangeles'}
producedict = {'APPLES':'apples','APRICOTS':'apricots','ASPARAGUS':'asparagus','AVOCADOS':'avocados',
               'BANANAS':'bananas','BEANS':'beans','BEETS':'beets','BLACKBERRIES':'blackberries',
               'BLUEBERRIES':'blueberries','BROCCOLI':'broccoli','CABBAGE':'cabbage','CANTALOUPS':'cantaloups',
               'CARROTS':'carrots','CAULIFLOWER':'cauliflower','CELERY':'celery','CHERRIES':'cherries',
               'CLEMENTINES':'clementines','CUCUMBERS':'cucumbers','ENDIVE':'endive','GARLIC':'garlic',
               'GINGER+ROOT':'ginger_root','GRAPEFRUIT':'grapefruit','GRAPES':'grapes','HONEYDEWS':'honeydews',
               'KIWIFRUIT':'kiwifruit','LEMONS':'lemons','LETTUCE%2C+ICEBERG':'iceberg_lettuce',
               'LETTUCE%2C+ROMAINE':'romaine_lettuce','LETTUCE%2C+RED+LEAF':'red_lettuce',
               'LETTUCE%2C+GREEN+LEAF':'green_lettuce','LIMES':'limes','NECTARINES':'nectarines',
               'OKRA':'okra','ORANGES':'oranges','PEACHES':'peaches','PEARS':'pears','PEAS+GREEN':'peas',
               'PEPPERS%2C+BELL+TYPE':'bell_peppers','PINEAPPLES':'pineapples','PLUMS':'plums',
               'POTATOES':'potatoes','RADISHES':'radishes','RASPBERRIES':'raspberries','RHUBARB':'rhubarb',
               'SPINACH':'spinach','SQUASH':'squash','STRAWBERRIES':'strawberries',
               'BRUSSELS+SPROUTS':'brussels_sprouts'}

cities = list(citydict.keys())
veggies = list(producedict.keys())

"""
for c in cities:
    for v in veggies:
        data = pd.read_csv(c+'_'+v+'_prophet_output.csv',index_col = 0)
        ## insert data into database from Python (proof of concept - this won't be useful for big data, of course)

        data.to_sql(citydict[c]+'_'+producedict[v], engine, if_exists='replace')
""" 
    
# test   
con = psycopg2.connect(database = dbname, user = username)

sql_query ="""
SELECT yhat, yhat_upper FROM "newyork_apples";
"""
query_results=pd.read_sql_query(sql_query,con)
print(query_results)

sql_query2 ="""
SELECT yhat, yhat_upper, ds FROM %(tablename)s;
"""
query_results=pd.read_sql_query(sql_query2,con,params={'tablename':AsIs('newyork_apples')})
print(query_results['ds'])

con.close()

           yhat  yhat_upper
0     19.725914   23.162578
1     19.828087   23.356355
2     19.866334   23.502175
3     19.906191   23.836940
4     19.947286   23.525083
5     19.989192   23.453975
6     20.114891   23.809727
7     20.155004   23.821795
8     20.193292   23.668237
9     20.229197   24.017449
10    20.262177   23.742581
11    20.338699   23.816395
12    20.355388   24.120819
13    20.367178   23.911242
14    20.373912   24.308897
15    20.375537   24.112617
16    20.350948   24.109327
17    20.333939   23.831928
18    20.313339   23.753943
19    20.289818   24.085537
20    20.264164   23.901990
21    20.183780   23.608783
22    20.159391   24.152432
23    20.109842   23.809595
24    20.120948   23.711556
25    20.143411   23.703489
26    20.176898   23.964006
27    20.222233   23.609607
28    20.280106   23.862617
29    20.533589   24.233686
...         ...         ...
2742  33.432951   36.922293
2743  33.642024   37.054237
2744  33.848747   37.486036
2745  34.052104   37